# Visualization
When it comes to visualizing geospatial data with/on maps with Python, a great number of tools and techniques
are available. In this lesson we will explore several of these:

* [Folium](#Folium)
* [ipyleaflet](#ipyleaflet)
* [Bokeh](#Bokeh)
* [pydeck](#pydeck)


## Folium
Whenever you visit a website that has some kind of interactive map, it
is quite probable that you are witnessing a map that has been made with
a JavaScript library called [Leaflet](http://leafletjs.com). Other popular libraries you may encounter are
[OpenLayers](https://openlayers.org) and [maplibre](https://maplibre.org/).

The Python module 
[Folium](https://github.com/python-visualization/folium) makes
it possible to visualize data that has been manipulated in Python on an
interactive Leaflet map in a jupyter notebook or website generated from python code.

### Basics
We will start with the most minimal map using the default OpenStreetMap base map.
See [Folium Quickstart](https://python-visualization.github.io/folium/quickstart.html).


In [1]:
import folium

folium_map = folium.Map(location=[-1.4563, -48.5013])

The `location` keyword argument (there are many more, with sensible defaults) provides the Map centre in Latitude and Longitude (Northing, Easting). To display the Map in a Jupyter notebook, simply ask for its object representation:


In [2]:
folium_map

You could even save this map to a file and serve it via a webserver: 


In [3]:
folium_map.save('test/07-folium-1.html')

Now [open this map here](test/07-folium-1.html) as "regular" HTML! 


### GeoJSON overlay
It gets interesting when you can overlay the map with data manipulated
via Python. Here we overlay the map with the Polygons of all countries, though
that set is in a lower resolution clearly.


In [4]:
countries = f'../data/countries.json'

folium_map2 = folium.Map(
    location=[0, 0],
    zoom_start=2  
)

folium.GeoJson(
    countries,
    name='countries'
).add_to(folium_map2)

folium.LayerControl().add_to(folium_map2)

folium_map2

In [5]:
# Also save this map
folium_map2.save('test/07-folium-2.html')

And [open this map with overlay here](test/07-folium-2.html). 



### Folium and Streamlit
Folium can also be [combined with Streamlit](https://folium.streamlit.app/). 
[Streamlit](https://streamlit.io) is a platform to create interactive web apps for your python data scripts.



## ipyleaflet
[ipyleaflet](https://ipyleaflet.readthedocs.io) provides similar functionality as folium, however because
it is based on [ipywidgets](https://ipywidgets.readthedocs.io), it integrates with other components from
the ipywidgets ecosystem (sliders, datagrids, tabs).

Links:

* GitHub: https://github.com/jupyter-widgets/ipyleaflet
* Documentation: https://ipyleaflet.readthedocs.io

### ipyleaflet - simplest map

In [6]:
from ipyleaflet import *

m = Map(center=(-1.4563, -48.5013), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)
m

Map(center=[-1.4563, -48.5013], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### ipyleaflet - add overlay tiles
This basemap is now transparently overlayed with tiles from the [Strava heatmap](https://www.strava.com/heatmap).

In [7]:
strava_all = basemap_to_tiles(basemaps.Strava.All)
m.add_layer(strava_all)
m

Map(center=[-1.4563, -48.5013], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### ipyleaflet - mouse interaction handling
We'll remove the overlay and add interaction to show lat/lon coordinates where your mouse is. 

In [8]:
m.remove_layer(strava_all)

from ipywidgets import Label

label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

m.on_interaction(handle_interaction)
m

Label(value='')

Map(center=[-1.4563, -48.5013], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### ipyleaflet - Add Overlay Vector Layer
We will add some rivers.


In [9]:
import json

with open('../data/rivers_lake_centerlines.json') as f:
    data = json.load(f)
    
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'blue',
        'weight': 1,
        'fillColor': 'blue',
        'fillOpacity': 0.5
    }
geo = GeoJSON(data=data, hover_style={'fillColor': 'red'}, name='Rivers-Lakes')
m.add_layer(geo)
m

Map(center=[-1.4563, -48.5013], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### ipyleaflet - Adding Control
Add a control to select layers to display on map.


In [10]:
m.add_control(LayersControl())

### ipyleaflet - Creating two maps side by side
As ipyleaflet is based on ipywidgets, we can make interesting combinations.


In [11]:
import ipywidgets
 
ipywidgets.HBox([m, Map(center=[-1.4563, -48.5013], zoom=8)])

### ipyleaflet - Adding Popups

#### 1. Adding a static popup

In [12]:
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Popup
center = (-1.4217175,-48.4921731)
m = Map(center=center, zoom=17, close_popup_on_click=False)
marker = Marker(location=(-1.4217175,-48.4921731))
m.add_layer(marker)
message2 = HTML()
message2.value = "Hey!! I'm speaking at FOSS4G 2024 🔥"
marker.popup = message2
m

Map(center=[-1.4217175, -48.4921731], close_popup_on_click=False, controls=(ZoomControl(options=['position', '…

#### 2. Using Custom data in popup

For this example we'll prepare map of following scenario
Seeing all the cities as a point on map and on click show their name

In [13]:
#Preparing data 
import geopandas as gpd
all_cities =  gpd.read_file('../data/shape/populated_places.shp')
all_countries =  gpd.read_file('../data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')
all_cities.dropna(subset=["NAME","geometry"])
Brazil = all_countries[all_countries['NAME'] == 'Brazil']
Brazilian_cities = all_cities[all_cities.within(Brazil.squeeze().geometry)]
Brazilian_cities

# Creating Map
from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML
center = (-15.749997,-47.9499962)
m = Map(center=center, zoom=3, close_popup_on_click=False)

# Adding data as marker 
for index, row in Brazilian_cities.iterrows():
    message2 = HTML()
    marker = Marker(location=(row['geometry'].y, row['geometry'].x))
    message2.value = row['NAME']
    # message2.description = row['NAME']
    marker.popup = message2
    m.add_layer(marker)
#     print(index)

#load map
m

Map(center=[-15.749997, -47.9499962], close_popup_on_click=False, controls=(ZoomControl(options=['position', '…

### Other interesting ipyleaflet functions

1. AntPath 
2. Marker Cluster
3. Heatmap
4. Velocity
5. Choropleth

check out out at https://ipyleaflet.readthedocs.io/

## Bokeh

Bokeh is a powerful framework to produce tailored interactive map and data visualisations.
Map features are limited compared to Folium, but there are more options to tailor the behaviour.
Bokeh provides mechanisms to interact with a server side application. With Geopandas and Bokeh
one can produce a nice looking interactive map like in the image below:

![Bokeh and Geopandas Example](images/bokeh-example1.jpg)
*Interactive Map with Bokeh and GeoPandas - Source: [CSC L6](https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html)*


### Bokeh - links

See also:

* https://automating-gis-processes.github.io/CSC/lessons/L6/interactive-map-bokeh.html
* [Binder for Geographic Plots in Bokeh](https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F09%20-%20Geographic%20Plots.ipynb)
* https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10
* https://pythonawesome.com/bokeh-plotting-backend-for-pandas-and-geopandas/


### Bokeh - making a simple plot
First, we learn the basic logic of plotting in Bokeh by making a simple interactive plot with a few points.

Import the necessary functionalities from Bokeh.

In [14]:
from bokeh.plotting import figure, save

Initialize our plot by calling the `figure` object.


In [15]:
p = figure(title='My first interactive plot!')

Next we create lists of x and y coordinates that we want to plot.


In [16]:
x_coords = [0,1,2,3,4]
y_coords = [5,4,1,2,0]

	In Bokeh drawing points, lines or polygons are always done using 
	list(s) of x and y coordinates.

Now we can plot those as points using a `.circle()` -object. Give it a red color and size of 10.


In [17]:
p.circle(x=x_coords, y=y_coords, size=10, color='red')

GlyphRenderer(id='p1044', ...)

Finally, we can save our interactive plot into the disk with save -function 
that we imported in the beginning. All interactive plots are typically 
saved as html files which you can open in a web-browser.
	
	# Save the plot by passing the plot -object and output path
	save(obj=p, filename="../data/output/07-points.html")

Now you could open your interactive `points.html` plot by double-clicking it which should open it in a web browser.

But we will plot directly in the Notebook here using `output_notebook()` and `show()`.


In [18]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

And then the moment of magic:


In [19]:
show(p)

### Bokeh - Creating an Interactive Tiled Background Map


In [20]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

If you show the figure, you can then use the wheel zoom and pan tools to navigate over any zoom level, 
and Bokeh will request the appropriate tiles from the server and insert them at the correct locations in the plot:


In [21]:
# When using in standard Python env
# output_file("tile.html")

# range bounds supplied in web mercator coordinates
p = figure(tools='pan, wheel_zoom', x_range=(-10000000, -3000000), y_range=(-6000000, 0),
           x_axis_type='mercator', y_axis_type='mercator')
p.add_tile("CartoDB Positron", retina=True)

show(p)

### Creating an Interactive Maps using Bokeh and Geopandas - OPTIONAL

Creating an interactive Bokeh map from a Shapefile or other vector data file like GeoJSON
consists typically of the following steps:

* Read the spatial vector file into `GeoDataFrame`
* Calculate the x and y coordinates of the geometries into separate columns
* Convert the `GeoDataFrame` into a Bokeh `DataSource`
* Plot the x and y coordinates as points, lines or polygons (which are in Bokeh words: `circle`, `multi_line` and `patches`)

We follow the steps below, extending and plotting on the tiled map from above.


In [22]:
import geopandas as gpd

# Read the data (already in Web Mercator projection (ignore the warning)
points = gpd.read_file('../data/populated_places.3857.gpkg')

In [23]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [24]:
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

In [25]:
points.head(5)

,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,DIFFASCII,NAMEASCII,ADM0CAP,...,name_nl,name_pl,name_sv,name_tr,name_vi,wdid_score,ne_id,geometry,x,y
0,8,10,3,Admin-0 capital,Vatican City,None,None,0,Vatican City,1.0,...,Vaticaanstad,Watykan,Vatikanstaten,Vatikan,Thành Vatican,4,1159127243,POINT (1386304.649 5146502.576),1.386305e+06,5.146503e+06
1,7,20,0,Admin-0 capital,San Marino,None,None,0,San Marino,1.0,...,San Marino,San Marino,San Marino,San Marino,Thành phố San Marino,4,1159146051,POINT (1385011.519 5455558.186),1.385012e+06,5.455558e+06
2,7,20,0,Admin-0 capital,Vaduz,None,None,0,Vaduz,1.0,...,Vaduz,Vaduz,Vaduz,Vaduz,Vaduz,4,1159146061,POINT (1059390.8 5963928.576),1.059391e+06,5.963929e+06
3,6,30,8,Admin-0 capital alt,Lobamba,None,None,0,Lobamba,0.0,...,Lobamba,Lobamba,Lobamba,Lobamba,Lobamba,4,1159146343,POINT (3473167.791 -3056995.457),3.473168e+06,-3.056995e+06
4,6,30,8,Admin-0 capital,Luxembourg,None,None,0,Luxembourg,1.0,...,Luxemburg,Luksemburg,Luxemburg,Lüksemburg,Luxembourg,4,1159146437,POINT (682388.791 6379291.915),6.823888e+05,6.379292e+06


In [26]:
p_df = points.drop('geometry', axis=1).copy()
p_df.head(2)

,SCALERANK,NATSCALE,LABELRANK,FEATURECLA,NAME,NAMEPAR,NAMEALT,DIFFASCII,NAMEASCII,ADM0CAP,...,name_ko,name_nl,name_pl,name_sv,name_tr,name_vi,wdid_score,ne_id,x,y
0,8,10,3,Admin-0 capital,Vatican City,None,None,0,Vatican City,1.0,...,바티칸 시국,Vaticaanstad,Watykan,Vatikanstaten,Vatikan,Thành Vatican,4,1159127243,1.386305e+06,5.146503e+06
1,7,20,0,Admin-0 capital,San Marino,None,None,0,San Marino,1.0,...,산마리노,San Marino,San Marino,San Marino,San Marino,Thành phố San Marino,4,1159146051,1.385012e+06,5.455558e+06


In [27]:
from bokeh.models import ColumnDataSource
psource = ColumnDataSource(p_df)

In [28]:
# p = figure(title="A map of populated places from a GeoPackage")
p.scatter('x', 'y', source=psource, color='lightblue',size=15, alpha=0.7)
show(p)

### Using Bokeh GeoJSONSource  - OPTIONAL
The above scenario could be effected even more compact.
The Bokeh `GeoJSONDataSource` expects a GeoJSON-string, so we can 
just use ordinary file `open()/read()`.

See `GeoJSONDataSource` example in the [Bokeh Documentation](https://bokeh.pydata.org/en/latest/docs/user_guide/geo.html#geojson-data).
Though we have to do reprojection from WGS84 (default for GeoJSON)
to EPSG:3857 (Web Mercator a.k.a. "Google" or "OSM" Projection) before. In this case we have already reprojected
the GeoJSON file using `ogr2ogr`:

	ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:3857 populated_places.3857.json populated_places.json


<div class="alert alert-block alert-warning">
<b>Warning:</b> Although GDAL/OGR, bokeh and many other tools support a crs member in GeoJSON, please note that the latest version of the <a href="https://datatracker.ietf.org/doc/html/rfc7946">GeoJSON spec<a> does not support specifying Coordinate Reference Systems and always assumes a default WGS84 crs. If you need crs support in GeoJSON, you consider the OGC <a href="https://github.com/opengeospatial/ogc-feat-geo-json">JSON-FG candidate Standard</a> from OGC.
</div>

In [29]:
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

# We could also assemble map plus overlay in an HTML file
# output_file("../data/output/07-bokeh-geojson.html")

Loading BokehJS ...

Read the populated places GeoJSON


In [30]:
with open('../data/populated_places.3857.json') as json_file:
        geojson = json_file.read()
        
geo_source = GeoJSONDataSource(geojson=geojson)

Build the map. Again, use the wheel zoom and pan tools to navigate the map. 


In [31]:
p = figure(tools='pan, wheel_zoom', x_axis_type='mercator', y_axis_type='mercator', width=800, height=500)

# add background tiles layer from CARTO
p.add_tile("CartoDB Positron", retina=True)

# add populated places point overlay
p.scatter(x='x', y='y', size=10, alpha=0.7, source=geo_source, color='lightblue', legend_label='Populated Places')

show(p)

## pydeck

Up till this point we've looked at 2D data visualisations. With [pydeck](https://pydeck.gl/)
we switch to WebGL-powered data visualization, including 3D and vector tiles.
pydeck is a python wrapper for the [deck.gl](https://deck.gl/) javascript library.
deck.gl visualisations typically use a vector tile background (from [mapbox](https://mapbox.com), [maptiler](https://www.maptiler.com/), or similar)

![Example by deck.gl](images/deck.gl.jpg)

The code snippet below creates a deck.gl view based on some sample data with tooltips.


In [32]:
import pydeck as pdk
import pandas as pd

# Sample data: 6 locations with population
data = pd.DataFrame({
    'lat': [43.5081, 43.2967, 43.0500, 43.3438, 42.6507, 42.9228],
    'lon': [16.4402, 17.0177, 17.4333, 17.8078, 18.0944, 17.6158],
    'pop': [160000 , 13000, 6500, 105000, 41000, 4000],
    'city': ['Split', 'Makarska', 'Ploče', 'Mostar', 'Dubrovnik', 'Neum']
})

# Define a scatterplot layer
layer = pdk.Layer(
    'ScatterplotLayer',
    data=data,
    get_position='[lon, lat]',
    get_radius='pop / 10',
    get_fill_color='[180, 0, 200, 140]',
    pickable=True
)

# Define map view
view_state = pdk.ViewState(
    latitude=42.7,
    longitude=16.8,
    zoom=7,
    pitch=30,
    bearing=10
)

# Render view
deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{city}"}
)

deck.show()

---
[<- Data analysis](06-data-analysis.ipynb) | [Metadata ->](08-metadata.ipynb)